In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.header import Header
from email.utils import formataddr
import glob
import time

In [ ]:
filepaths = glob.glob('personaldata/*')
filepaths

In [ ]:
# --- 邮件配置 ---
# SMTP 服务器配置
smtp_server = 'smtp.qq.com'  # 例如: smtp.qq.com
smtp_port = 465              # SSL 端口
# 发件人账户信息
sender_email = '123456789@qq.com'      # 你的邮箱地址
sender_password = 'abcdefghijklmn'     # 你的邮箱授权码，而不是登录密码！


In [ ]:
server = None
try:
    print("正在连接到 SMTP 服务器...")
    server = smtplib.SMTP_SSL(smtp_server, smtp_port, timeout=20)
    print("服务器连接成功，正在登录...")
    server.login(sender_email, sender_password)
    print("登录成功，准备开始批量发送邮件...\n")

    for filepath in filepaths:
        try:
            number = filepath.split(sep="\\")[1].split(".")[0]
            number = str(number)
            print("______________________________________________")
            print(f"正在为学号 {number} 准备邮件...")

            recipient_email = f'{number}@stu.sufe.edu.cn'

            msg = MIMEMultipart()
            msg['From'] = formataddr((Header('倪伟丰', 'utf-8').encode(), sender_email))
            msg['To'] = formataddr((Header(f'{number}', 'utf-8').encode(), recipient_email))
            msg['Subject'] = Header('综测评分数据，请检查是否计算正确', 'utf-8')
            body_content = "你好，这是你的综测评分数据，请你检查是否有计算错误！"
            msg.attach(MIMEText(body_content, 'plain', 'utf-8'))

            with open(filepath, 'rb') as f:
                attachment = MIMEApplication(f.read())
            attachment.add_header('Content-Disposition', 'attachment', filename=filepath)
            msg.attach(attachment)

            server.sendmail(sender_email, [recipient_email], msg.as_string())
            print(f"邮件已成功发送至: {recipient_email}")

        except FileNotFoundError:
            print(f"[错误] 附件文件未找到于路径 '{filepath}'，已跳过。")
            continue
        except Exception as e:
            print(f"[错误] 处理 {number} 的邮件时发生错误: {e}，已跳过。")
            continue

        time.sleep(1)

except smtplib.SMTPAuthenticationError:
    print("SMTP 认证失败！请检查邮箱地址和授权码是否正确。")
except Exception as e:
    print(f"发生严重错误: {e}")
finally:
    if server:
        try:
            server.quit()
            print("\n所有邮件处理完毕，已成功关闭与服务器的连接。")
        except smtplib.SMTPServerDisconnected:
            print("\n服务器已提前断开连接，程序正常清理完毕。")